In [1]:
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

In [2]:
root_saving_folder = Path("/neurospin/dico/tsanchez/Test_BetaVAE/2025-04-22")
timestamp = "13-45-45"
name_checkpoint = "checkpoint.pt"

path_model = root_saving_folder / timestamp / name_checkpoint

In [3]:
def adjust_in_shape(in_shape, depth):
    """
    Function to make sure that the output of the encoder is composed of integers 
    In this case : Each block (conv_x + conv_x_a) reduce by 2 the dimension of the volume.
    """

    dims=[]
    for idx in range(1, 4):
        dim = in_shape[idx]
        r = dim%(2**depth)
        if r!=0:
            dim+=(2**depth-r)
        dims.append(dim)
    return((1, dims[0]+4, dims[1], dims[2])) 

In [4]:
from betaVAE.beta_vae import VAE

device = "cuda:0"
DEPTH = 3

IN_SHAPE_WOUT_ADJUST = [1, 54, 120, 139] #One from the config.yaml file
IN_SHAPE = adjust_in_shape(IN_SHAPE_WOUT_ADJUST, depth=DEPTH)

# ! Specific to the different models
N_LATENT = 64 
model = VAE(
    in_shape=IN_SHAPE, 
    n_latent=N_LATENT,
    depth=DEPTH
)

model.load_state_dict(torch.load(path_model))
model.to(device)
# model.eval()

/tmp/ipykernel_2779551/3755725018.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path_model))


VAE(
  (encoder): Sequential(
    (conv0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (norm0): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (LeakyReLU0): LeakyReLU(negative_slope=0.01)
    (conv0a): Conv3d(16, 16, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (norm0a): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (LeakyReLU0a): LeakyReLU(negative_slope=0.01)
    (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (norm1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (LeakyReLU1): LeakyReLU(negative_slope=0.01)
    (conv1a): Conv3d(32, 32, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (norm1a): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (LeakyReLU1a): LeakyReLU(negative_slope=0.01)
    (conv2): Conv3d(32, 64, kernel_size=(3, 3

In [5]:
from betaVAE.preprocess import UkbDataset
config = {"in_shape" : IN_SHAPE,
          "root" : "/neurospin/dico/tsanchez/preprocessed/UKBio1000" }
dataset = UkbDataset(config)

ModuleNotFoundError: No module named 'omegaconf'

In [23]:
SUBJECT_IND = 42
tensor, sub_id = dataset[SUBJECT_IND]
tensor = tensor.to(device= device, dtype = torch.float32).unsqueeze(0)
sub_id, tensor.shape, torch.unique(tensor)

('sub-2614736',
 torch.Size([1, 1, 60, 120, 144]),
 tensor([-1.,  0.,  1.], device='cuda:0'))

In [24]:
output, mean, logvar = model(tensor)
output, output.shape

(tensor([[[[[ 1.3208e-01,  1.3483e-01,  1.4111e-01,  ...,  1.4068e-01,
              1.3751e-01,  1.2757e-01],
            [ 1.4849e-01,  1.6506e-01,  1.5217e-01,  ...,  2.0958e-01,
              1.5025e-01,  2.0253e-01],
            [ 1.3577e-01,  1.3803e-01,  1.3631e-01,  ...,  1.5008e-01,
              1.3382e-01,  1.3652e-01],
            ...,
            [ 1.4228e-01,  1.6233e-01,  1.5810e-01,  ...,  1.9755e-01,
              1.4937e-01,  2.1839e-01],
            [ 1.2697e-01,  1.3289e-01,  1.2972e-01,  ...,  1.4054e-01,
              1.3040e-01,  1.3113e-01],
            [ 1.3699e-01,  1.2945e-01,  1.4149e-01,  ...,  1.5074e-01,
              1.3854e-01,  1.3471e-01]],
 
           [[ 1.7632e-01,  1.4671e-01,  2.1302e-01,  ...,  1.6143e-01,
              2.1969e-01,  1.3753e-01],
            [ 1.3194e-01,  1.6447e-01,  1.5202e-01,  ...,  1.9515e-01,
              1.4515e-01,  1.8225e-01],
            [ 2.3020e-01,  1.6599e-01,  2.5777e-01,  ...,  1.9003e-01,
              2.3867e

In [25]:
softmax = torch.nn.Softmax(dim= 1)
output_soft = softmax(output)
agmax_ouput = torch.argmax(output_soft, dim = 1)
agmax_ouput.shape, torch.unique(agmax_ouput)

(torch.Size([1, 60, 120, 144]), tensor([0, 1], device='cuda:0'))

In [26]:
agmax_ouput.to(device = "cpu")

tensor([[[[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         ...,

         [[1, 1, 1,  ..., 1, 1, 1],
          [0, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [0, 1, 1,  ..., 1, 1, 1],
          [1, 0, 1,  ..., 1, 1, 1],
          [0, 1, 1,  ..., 1, 1, 1]],

         [[0, 1, 1,  ..., 1, 1, 1],
          [1, 1, 

In [28]:
import numpy as np
root_tmp = Path("/neurospin/dico/tsanchez/tmp/")

np.save(root_tmp / "output_inference.npy", agmax_ouput.cpu().numpy())
np.save(root_tmp / "input_inference.npy", tensor.squeeze(0).cpu().numpy())

In [10]:
wm_and_vermis = tensor[0][0][20:40][:][:].cpu().numpy()
slice_output = agmax_ouput[0][20:40][:][:].cpu().numpy()
np.unique(wm_and_vermis), wm_and_vermis.shape, np.unique(slice_output), slice_output.shape

(array([-1.,  0.,  1.], dtype=float32),
 (20, 120, 144),
 array([0, 1]),
 (20, 120, 144))

In [11]:
white_matter = np.where(wm_and_vermis == -1, 1, 0)
sulci = np.where(wm_and_vermis == 1, 1, 0)
